<a href="https://colab.research.google.com/github/MatheusCascalho/RoboticSimulationLab/blob/main/tpUaibot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import uaibot as ub
import numpy as np
import random
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

###########################################
# Global variables

#Initial positions
htm_brush_box = ub.Utils.trn([0.4, 0.4, 0.1])
htm_robot_init = ub.Utils.trn([0.35,0,0.8])*ub.Utils.roty(np.pi/2)
htm_brush_box_target = ub.Utils.trn([0.4, 0.4, 0.2])*ub.Utils.roty(np.pi)

#Creating objects
brush_box = ub.Box(color='yellow',width=0.1,depth=0.1,height=0.2,opacity=0.5)
robot = ub.Robot.create_kuka_lbr_iiwa()
frame = ub.Frame(htm_brush_box_target, size=0.1)

objects = [robot,frame, brush_box]
matrix = []

#Constants
dt = 0.05
K = 0.8
n_joins = np.shape(robot.q)[0]
n = 3
m = 3
a=0.25
wtol = 0.01
caminho = "/content/paisagem.jpg"

def carregar_imagem(caminho_imagem):
    return Image.open(caminho_imagem).convert("RGB")

def reduzir_cores(imagem, num_cores=3):
    img_np = np.array(imagem)
    h, w, _ = img_np.shape
    img_flat = img_np.reshape((-1, 3))

    kmeans = KMeans(n_clusters=num_cores, n_init=10)
    kmeans.fit(img_flat)

    cores = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_

    img_reduzida = cores[labels].reshape((h, w, 3)).astype(np.uint8)
    return Image.fromarray(img_reduzida)

def redimensionar_imagem(imagem, n, m):
    return imagem.resize((m, n), Image.NEAREST)

def imagem_para_matriz_hex(imagem):
    img_np = np.array(imagem)
    matriz_hex = []

    for linha in img_np:
        linha_hex = ['#{:02x}{:02x}{:02x}'.format(*pixel) for pixel in linha]
        matriz_hex.append(linha_hex)
    return matriz_hex

def create_board(n, board_colors):
  size = 0.4/n
  dt_=0
  for j in range(n):
    for i in range(n):
      htm_pixel = ub.Utils.trn([0.5+0.01, -0.2+size/2+(i)*size,1.0-size/2-(j)*size])*ub.Utils.rotz(np.pi/2)
      pixel = ub.Box(color='black',width=size,depth=0.01,height=size,opacity=0.9)
      pixel.add_ani_frame(time=dt_, htm=htm_pixel)
      matrix.append(pixel)
      dt_ = dt_ + 0.0001

  objects.append(matrix)

def move_to_initial_position(robot):
  robot.add_ani_frame(time=0, q=robot.ikm(htm_robot_init))

def task_function(_q, _htm_d):
  Jg, htm = robot.jac_geo(_q)

  jac_v = Jg[0:3,:]
  jac_w = Jg[3:6,:]

  Se = htm[0:3, 3]
  xe = htm[0:3, 0]
  ye = htm[0:3, 1]
  ze = htm[0:3, 2]

  Sd = _htm_d[0:3, 3]
  xd = _htm_d[0:3, 0]
  yd = _htm_d[0:3, 1]
  zd = _htm_d[0:3, 2]

  r = np.matrix(np.zeros((6, 1)))
  r[0:3] = Se-Sd
  r[3] = 1 - xd.T*xe
  r[4] = 1 - yd.T*ye
  r[5] = 1 - zd.T*ze


  Jr = np.matrix(np.zeros((6, n_joins)))
  Jr[0:3,:] = jac_v
  Jr[3,:] = xd.T * ub.Utils.S(xe) * jac_w
  Jr[4,:] = yd.T * ub.Utils.S(ye) * jac_w
  Jr[5,:] = zd.T * ub.Utils.S(ze) * jac_w

  return r, Jr

def fun_F(r):
    A = [a, a, a, a, a, a]
    w_tol = [wtol, wtol, wtol, wtol, wtol, wtol]
    F = np.matrix(np.zeros((6, 1)))
    for i in range(6):
        if abs(r[i, 0]) < w_tol[i]:
            F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
        elif r[i, 0] >= w_tol[i]:
            F[i, 0] = -A[i]
        else:
            F[i, 0] = A[i]
    return F

def move_to_position(htm_target, t0):
  # print("movendo para a posição: ")
  # print(htm_target)
  # error = True
  # while (error != False):
  i = 0
  while(True):
    t = dt*i + t0
    r , Jr = task_function(robot.q, htm_target)

    q_dot_d = ub.Utils.dp_inv(Jr,0.001)*fun_F(r)
    q_next = robot.q + q_dot_d*dt

    # (ok, msg, _) = robot.check_free_configuration(q = q_next)
    # if(~ok):
    #   print('DEU RUIM')
    #   print(msg)
    #   error = True
    #   break


    robot.add_ani_frame(time=t,q=q_next)
    i = i+1
    if(np.linalg.norm(r) < 1e-2):
      #error = False
      break

    if(dt*i > 5):
      #error = True
      #print("Não foi possível alcançar a posição:")
      #print(htm_target)
      #print("Erro atual é de:")
      #print(np.linalg.norm(r))
      break

  return t

imagem = carregar_imagem(caminho)
imagem_reduzida = reduzir_cores(imagem, num_cores=6)
imagem_redimensionada = redimensionar_imagem(imagem_reduzida, n, m)
board_colors = imagem_para_matriz_hex(imagem_redimensionada)

create_board(n, np.transpose(board_colors)) # Cria nxn pixels na tela


#Setting initial positions
brush_box.add_ani_frame(time=0, htm=htm_brush_box)
move_to_initial_position(robot) # Consertar para que fique alinhado

t = 0
t = move_to_position(htm_brush_box_target, t)
t = move_to_position(htm_robot_init, t)

htm_v1 = ub.Utils.trn([0.5, 0, 0.8])*ub.Utils.roty(np.pi/2);
t = move_to_position(htm_v1, t)

htm_v1 = ub.Utils.trn([0.5, 0.2, 1.0])*ub.Utils.roty(np.pi/2);
t = move_to_position(htm_v1, t)
htm_v1 = ub.Utils.trn([0.5, -0.2, 1.0])*ub.Utils.roty(np.pi/2);
t = move_to_position(htm_v1, t)
htm_v1 = ub.Utils.trn([0.5, -0.2, 0.6])*ub.Utils.roty(np.pi/2);
t = move_to_position(htm_v1, t)
htm_v1 = ub.Utils.trn([0.5, 0.2, 0.6])*ub.Utils.roty(np.pi/2);
t = move_to_position(htm_v1, t)
htm_v1 = ub.Utils.trn([0.5, 0.2, 1.0])*ub.Utils.roty(np.pi/2);
t = move_to_position(htm_v1, t)
size = 0.4/n
for j in range(n):
  for i in range(n):
    if(np.transpose(board_colors)[i][j] == '#ffffff'):
      y_pixel = -0.2+size/2+(i)*size
      z_pixel = 1.0-size/2-(j)*size
      htm_pixel = ub.Utils.trn([0.5+0.01, y_pixel,z_pixel])*ub.Utils.rotz(np.pi/2)
      pixel = ub.Box(color=np.transpose(board_colors)[i][j],width=size,depth=0.01,height=size,opacity=0.9)
      pixel.add_ani_frame(time=t-0.1, htm=htm_pixel)
      #matrix.append(pixel)
      objects.append(pixel)
      i=i+1

    if (np.transpose(board_colors)[i][j] == np.transpose(board_colors)[i-1][j]):
      y_pixel = -0.2+size/2+(i)*size
      z_pixel = 1.0-size/2-(j)*size
      htm_pixel = ub.Utils.trn([0.5+0.01, y_pixel,z_pixel])*ub.Utils.rotz(np.pi/2)
      htm_init_paint = htm_pixel*ub.Utils.trn([0, 0, size/2])*ub.Utils.rotx(np.pi/2);
      htm_finish_paint = htm_pixel*ub.Utils.trn([0, 0, -size/2])*ub.Utils.rotx(np.pi/2);
      pixel = ub.Box(color=np.transpose(board_colors)[i][j],width=size,depth=0.01,height=size,opacity=0.9)
      #matrix.append(pixel)
      objects.append(pixel)
      t = move_to_position(htm_init_paint, t)
      pixel.add_ani_frame(time=t+1, htm=htm_pixel)
      t = move_to_position(htm_finish_paint, t)


    if(np.transpose(board_colors)[i][j] != np.transpose(board_colors)[i-1][j]):
      t = move_to_position(htm_robot_init, t)
      t = move_to_position(htm_brush_box_target, t)
      y_pixel = -0.2+size/2+(i)*size
      z_pixel = 1.0-size/2-(j)*size
      htm_pixel = ub.Utils.trn([0.5+0.01, y_pixel,z_pixel])*ub.Utils.rotz(np.pi/2)
      htm_init_paint = htm_pixel*ub.Utils.trn([0, 0, size/2])*ub.Utils.rotx(np.pi/2);
      htm_finish_paint = htm_pixel*ub.Utils.trn([0, 0, -size/2])*ub.Utils.rotx(np.pi/2);
      pixel = ub.Box(color=np.transpose(board_colors)[i][j],width=size,depth=0.01,height=size,opacity=0.9)
      #matrix.append(pixel)
      objects.append(pixel)
      t = move_to_position(htm_robot_init, t)
      t = move_to_position(htm_init_paint, t)
      pixel.add_ani_frame(time=t+1, htm=htm_pixel)
      t = move_to_position(htm_finish_paint, t)





sim = ub.Simulation(objects, background_color="#AAAAAA")
sim.run()

In [ ]:
#!pip uninstall -y numpy scipy uaibot scikit-learn
#!pip install numpy scipy uaibot scikit-learn pillow matplotlib
!pip install uaibot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 33.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.7 MB/s eta 0:00:00
  Created wheel for uaibot: filename=uaibot-1.2.6-cp311-cp311-linux_x86_64.whl size=9464201 sha256=e13157d11b9e2d6c0462f44f6ff809b397fc8beeb18c475c2d3378c978bdd998
  Stored in directory: /root/.cache/pip/wheels/d7/79/71/4ff8e470e1634e8b5e93ac4410ad145861630004a27b6eaece
Successfully built uaibot
